In [101]:
import sys, os
import netCDF4
import scipy.io, numpy
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.patches as mpatches
import re

## Functions

In [102]:
# generate k-pdf plots
def plot_k_pdf(htdma_files):
    xnew = np.linspace(0, 1.3, 30) # K0
    df_k = pd.DataFrame(columns=['dia', 'K0', 'dNdK'])

    for htdma_file in htdma_files:      
    # =============== FOR EACH FILE ===============
    # get dimensions of time and bins
        filename = f'/Users/evenhou/Downloads/HOU_data/{htdma_file}'
        ds = netCDF4.Dataset(filename)
        s1 = str(ds.dimensions['time'])
        s2 = str(ds.dimensions['bin'])
        # print(s)
        x = -3
        if (s1[x-1:x+1] != '='):
            dim_time = int(s1[x:])
        else:
            dim_time = int(s1[x-1:])
        if (s2[x-1:x+1] != '='):
            dim_bin = int(s2[x:])
        else:
            dim_bin = int(s2[x-1:])

# CHANGE HERE: define arrays
        time = np.zeros(dim_time)
        dry_dia = np.zeros(dim_time)
        kappa = np.zeros((dim_time, dim_bin))
        k_bound = np.zeros((dim_time, dim_bin,2))
        conc = np.zeros((dim_time, dim_bin))
        
# CHANGE HERE: read data into arrays
        ncf = scipy.io.netcdf_file(filename, mmap=False)
        date = int(filename[-18:-10]) # read i.e. 20210427
        time[:] = ncf.variables["time"].data/3600 # hr
        dry_dia[:] = ncf.variables["dry_diameter_setting"].data # nm
        for i_time in range(dim_time):
            kappa[i_time, :] = ncf.variables['kappa'][i_time, :]
            k_bound[i_time, :, :] = ncf.variables['kappa_bounds'][i_time, :, :]
            conc[i_time,:] = ncf.variables['aerosol_concentration'][i_time, :] # dN, unit: 1/cm^3

# calculations
        dK = np.zeros((dim_time,dim_bin))
        for i_time in range(dim_time):
            dK[i_time, :] = k_bound[i_time, :, 1] -  k_bound[i_time, :, 0]
        conc_norm0 = np.zeros((dim_time,dim_bin)) # dN/dK
        for i_time in range(dim_time):
            conc_norm0[i_time, :] = conc[i_time, :]/dK[i_time, :]
        
# use pandas to manipulate data
        d = {'dry_dia': dry_dia, 'time':time, 'date':date}
        df = pd.DataFrame(data = d)
        #print(df)

        df_conc = pd.DataFrame(conc)
        # print(df_conc)
        N_tot = df_conc.sum(axis=1)# N_tot (series) for each scan
        N_tot.name = "N_tot"
        #print(N_tot) 
        #print(type(N_tot)) # Series
        
        df = df.join(N_tot)
        # print(df)
        Ntot = N_tot.to_numpy()
        conc_norm = np.zeros((dim_time,dim_bin))
        for i_time in range(dim_time):
            conc_norm[i_time, :] = conc_norm0[i_time, :]/Ntot[i_time]
        # print(conc_norm)

        dNdK = np.zeros((dim_time,30))
        for i_time in range(dim_time):
        # i_time = 1
            # dia = dry_dia[i_time]
            x = kappa[i_time,:] # one scan:kappa
            y = conc_norm[i_time,:] # one scan: c(k)
            f = interpolate.interp1d(x,y,fill_value="extrapolate", kind='nearest')
            ynew = f(xnew)
            dNdK[i_time,:] = ynew
        dia = np.repeat(dry_dia, 30) # for one scan
        x_flat = np.tile(xnew, dim_time) # for different bin
        # print(x_flat)
        y_flat = dNdK.ravel()
        # print(y_flat.shape)

        temp_df = pd.DataFrame({'dia': dia,'K0': x_flat,'dNdK': y_flat})
        df_k = pd.concat([df_k, temp_df], ignore_index=True)

        def k_size(df_k,size):
            if size == 50:
                df_k_i = df_k[df_k["dia"] == 50]  
            elif size == 100:
                df_k_i = df_k[df_k["dia"] == 100]
            elif size == 150:
                df_k_i = df_k[df_k["dia"] == 150]
            elif size == 200:
                df_k_i = df_k[df_k["dia"] == 200]  
            elif size == 250:
                df_k_i = df_k[df_k["dia"] == 250]  

            dNdK_i = df_k_i['dNdK'].to_numpy()
            d_scan = int(df_k_i.shape[0]/30)
            dNdK_i_matrix = dNdK_i.reshape(d_scan, 30)
            # print(np.isnan(dNdK100_matrix).any()) 
            dNdK_i_matrix = np.nan_to_num(dNdK_i_matrix) # could result in inaccuracy -> should remove this data when calculating the average
            # print(dNdK100_matrix.shape)
            avg_i = np.mean(dNdK_i_matrix, axis=0)

            return df_k_i.K0, df_k_i.dNdK, avg_i
        
        x50, y50, avg50 = k_size(df_k,50)
        x100, y100, avg100 = k_size(df_k,100)
        x150, y150, avg150 = k_size(df_k,150)
        x200, y200, avg200 = k_size(df_k,200)
        x250, y250, avg250 = k_size(df_k,250)

        date_str = f"{date:8}"  # Convert to string for formatting
        formatted_date = f"{date_str[0:4]}-{date_str[4:6]}-{date_str[6:8]}"
        plt.figure(figsize=(5,4))
        plt.plot(xnew,avg50, label='50 cm', c='tab:red')
        plt.plot(xnew,avg100, label='100 cm', c='tab:green')
        plt.plot(xnew,avg150, label='150 cm', c='tab:blue')
        plt.plot(xnew,avg200, label='200 cm', c='tab:purple')
        plt.plot(xnew,avg250, label='250 cm', c='tab:orange')
        plt.ylim(0,10)
        plt.legend()
        plt.title(f'Κ-pdf for all diameters on {formatted_date}')
        plt.xlabel('Κ')
        plt.ylabel('pdf')
        # plt.show()

        plot_path = f'/Users/evenhou/Documents/images/{formatted_date}_kpdf.png'
        plot_name = f'{formatted_date}_kpdf.png'
         # Save the plot
        plt.savefig(plot_path)
        # print(plot_name)
        # Close the plot to avoid display
        plt.close()
        #print(plot_name)

    # return plot_path

In [103]:
def plot_acsm(acsm_files):
    for acsm_file in acsm_files:
        filename = f'/Users/evenhou/Downloads/HOU_data/{acsm_file}'
        ds = netCDF4.Dataset(filename)
        s1 = ds.dimensions['time']
        dim_time = s1.size
        
        # define arrays
        time = np.zeros(dim_time)
        total_organics_i = np.zeros(dim_time)
        sulfate_i = np.zeros(dim_time)
        ammonium_i = np.zeros(dim_time)
        nitrate_i = np.zeros(dim_time)
        chloride_i = np.zeros(dim_time)
        total_conc_i = np.zeros(dim_time)

        ncf = scipy.io.netcdf_file(filename, mmap=False)
        date = int(filename[-18:-10])
        

        time[:] = ncf.variables["time"].data/3600 # hr
        total_organics_i[:] = ncf.variables["total_organics_CDCE"].data # ug/m^3
        sulfate_i[:] = ncf.variables["sulfate_CDCE"].data # ug/m^3
        ammonium_i[:] = ncf.variables["ammonium_CDCE"].data # ug/m^3
        nitrate_i[:] = ncf.variables["nitrate_CDCE"].data # ug/m^3
        chloride_i[:] = ncf.variables["chloride_CDCE"].data # ug/m^3  
        total_conc_i[:] = total_organics_i + sulfate_i + ammonium_i + nitrate_i + chloride_i

        plt.figure(figsize=(5,4))
        plt.plot(time,total_organics_i, label='total organics', c='tab:green',marker='o')
        plt.plot(time,sulfate_i, label='sulfate', c='tab:red',marker='o')
        plt.plot(time,ammonium_i, label='ammonium', c='darkorange',marker='o')
        plt.plot(time,nitrate_i, label='nitrate', c='tab:blue',marker='o')
        plt.plot(time,chloride_i, label='chloride', c='tab:purple',marker='o')
        plt.plot(time,total_conc_i, label='total concentration', c='black',marker='o')
        date_str = f"{date:8}"  # Convert to string for formatting
        formatted_date = f"{date_str[0:4]}-{date_str[4:6]}-{date_str[6:8]}"  # YYYY-MM-DD
        plt.title(f'ACSM Mass Concentration Data on {formatted_date}')
        # plt.xlabel('Time')
        plt.ylabel('Mass Concentration(ug/m^3)')
        plt.legend()

        plot_path = f'/Users/evenhou/Documents/images/{formatted_date}_acsm.png'
        plot_name = f'{formatted_date}_acsm.png'
         # Save the plot
        plt.savefig(plot_path)

        # plt.show()
        plt.close()
    
        #print(plot_name)

In [104]:
def plot_chi(htdma_files):
    for htdma_file in htdma_files:      
    # =============== FOR EACH FILE ===============
    # get dimensions of time and bins
        filename = f'/Users/evenhou/Downloads/HOU_data/{htdma_file}'
        ds = netCDF4.Dataset(filename)
        s1 = str(ds.dimensions['time'])
        s2 = str(ds.dimensions['bin'])
        # print(s)
        x = -3
        if (s1[x-1:x+1] != '='):
            dim_time = int(s1[x:])
        else:
            dim_time = int(s1[x-1:])
        if (s2[x-1:x+1] != '='):
            dim_bin = int(s2[x:])
        else:
            dim_bin = int(s2[x-1:])
        
        kappa_LH = 0.01
        kappa_MH = 0.65

        chi50 = np.array([]) 
        chi100 = np.array([])
        chi150 = np.array([])
        chi200 = np.array([])
        chi250 = np.array([])

# CHANGE HERE: define arrays
        time = np.zeros(dim_time)
        dry_dia = np.zeros(dim_time)
        kappa = np.zeros((dim_time, dim_bin))
        k_bound = np.zeros((dim_time, dim_bin,2))
        conc = np.zeros((dim_time, dim_bin))
        # date = np.zeros(dim_time)
        
# CHANGE HERE: read data into arrays
        ncf = scipy.io.netcdf_file(filename, mmap=False)
        date = int(filename[-18:-10]) # read i.e. 20210427
    
        time[:] = ncf.variables["time"].data/3600 # hr
        dry_dia[:] = ncf.variables["dry_diameter_setting"].data # nm
        for i_time in range(dim_time):
            kappa[i_time, :] = ncf.variables['kappa'][i_time, :]
            k_bound[i_time, :, :] = ncf.variables['kappa_bounds'][i_time, :, :]
            conc[i_time,:] = ncf.variables['aerosol_concentration'][i_time, :] # dN, unit: 1/cm^3
        # print(dry_dia)
        # print(k_bound)

# calculations
        dK = np.zeros((dim_time,dim_bin))
        for i_time in range(dim_time):
            dK[i_time, :] = k_bound[i_time, :, 1] -  k_bound[i_time, :, 0]
        conc_norm0 = np.zeros((dim_time,dim_bin)) # dN/dK
        for i_time in range(dim_time):
            conc_norm0[i_time, :] = conc[i_time, :]/dK[i_time, :]
        
        # Create a mask for valid kappa values
        valid_kappa_mask = (kappa >= kappa_LH) & (kappa <= kappa_MH)
        date_str = f"{date:8}"  # Convert to string for formatting
        formatted_date = f"{date_str[0:4]}-{date_str[4:6]}-{date_str[6:8]}"  # YYYY-MM-DD
        # Update conc matrix for invalid kappa values
        conc[~valid_kappa_mask] = 0
        N_tot = conc.sum(axis=1)
        # Check if there is 0 value in N_tot
        has_zero = np.any(N_tot == 0)
        #print(has_zero)
        # Find indices of zero values
        zero_indices = np.where(N_tot == 0)
        #print(zero_indices)
        # use pandas to manipulate data
        d = {'dry_dia': dry_dia, 'time':time, 'date':date, 'N_tot':N_tot}
        df = pd.DataFrame(data = d)
        conc_norm = np.zeros((dim_time,dim_bin))
        for i_time in range(dim_time):
            conc_norm[i_time, :] = conc_norm0[i_time, :]/N_tot[i_time]
        
        newdf = pd.DataFrame(np.repeat(df.values, dim_bin, axis=0),
                                columns=df.columns)
        # print(dim_bin)
        #print(newdf.shape)
        newdf['kappa'] = kappa.ravel() # K_i
        newdf['conc'] = conc.ravel()
        newdf['dK'] = dK.ravel()
        newdf['conc_norm0'] = conc_norm0.ravel()  # dN/dK
        rows_with_nan = np.where(np.isnan(conc_norm).any(axis=1))[0]
        #print("rows_with_nan",rows_with_nan)

        dNdK0 = np.array([])
        # Remove corresponding rows from both conc_norm, kappa, and dry_dia
        conc_norm_clean = np.delete(conc_norm, rows_with_nan, axis=0)
        kappa_clean = np.delete(kappa, rows_with_nan, axis=0)
        dry_dia_clean = np.delete(dry_dia, rows_with_nan, axis=0)
        time_clean = np.delete(time, rows_with_nan, axis=0)
        dNdK0 = np.append(dNdK0,conc_norm_clean)

        newdf['conc_norm'] = newdf['conc_norm0']/newdf['N_tot'] # dN°/dK = c(k)_i
        newdf['p_MH_i'] = (newdf.kappa-kappa_LH)/(kappa_MH-kappa_LH)
        newdf['p_LH_i'] = 1 - newdf['p_MH_i']

        newdf.loc[newdf['N_tot'] == 0, 'p_MH_i'] = 0
        newdf.loc[newdf['kappa'] < kappa_LH, 'p_MH_i'] = 0
        newdf.loc[newdf['kappa'] > kappa_MH, 'p_MH_i'] = 0

        newdf.loc[newdf['N_tot'] == 0, 'p_LH_i'] = 0
        newdf.loc[newdf['kappa'] < kappa_LH, 'p_LH_i'] = 0
        newdf.loc[newdf['kappa'] > kappa_MH, 'p_LH_i'] = 0

        newdf['H_i'] = -newdf.p_LH_i*np.log(newdf.p_LH_i)-newdf.p_MH_i*np.log(newdf.p_MH_i)
        newdf['H_alpha_i'] = newdf.H_i*newdf.conc_norm*newdf.dK
        newdf['p_mh_i'] = newdf.p_MH_i*newdf.conc_norm*newdf.dK
        newdf['p_lh_i'] = newdf.p_LH_i*newdf.conc_norm*newdf.dK

        newdf.loc[newdf['N_tot'] == 0, 'H_alpha_i'] = 0
        newdf.loc[newdf['kappa'] < kappa_LH, 'H_alpha_i'] = 0
        newdf.loc[newdf['kappa'] > kappa_MH, 'H_alpha_i'] = 0

        H_alpha = np.array([])
        p_MH = np.array([])
        p_LH = np.array([])
        for i in range(0,int(len(newdf)/60)):
            h_alpha = (newdf.iloc[i*60:i*60+59].sum()).H_alpha_i
            H_alpha = np.append(H_alpha,h_alpha)
            p_mh = (newdf.iloc[i*60:i*60+59].sum()).p_mh_i
            p_lh = (newdf.iloc[i*60:i*60+59].sum()).p_lh_i
            p_MH = np.append(p_MH,p_mh)
            p_LH = np.append(p_LH,p_lh)

        D_alpha = np.exp(H_alpha)
        # print(D_alpha)
        H_gamma = -p_LH*np.log(p_LH)-p_MH*np.log(p_MH)
        # print("H_gamma",H_gamma)

        H_gamma = np.nan_to_num(H_gamma)
        # print("H_gamma",H_gamma)

        D_gamma = np.exp(H_gamma)
        chi = (D_alpha-1)/(D_gamma-1)
                    # print("chi",chi)
                    # print(len(chi))

        df['p_MH'] = p_MH.tolist()
        df['p_LH'] = p_LH.tolist()
        df['D_alpha'] = D_alpha.tolist()
        df['D_gamma'] = D_gamma.tolist()
        df['chi'] = chi.tolist()
        # plt.scatter(time,p_LH)
        # print(df)

        df_valid = df[df['N_tot'] != 0]
        # print(df_valid.chi)
        df_valid = df.dropna(subset=["chi"])
        # print(df_valid.chi)
        # print(df_valid)
        chi_final = df_valid["chi"].to_numpy()

        df50 = df_valid[df_valid["dry_dia"] == 50]
        df100 = df_valid[df_valid["dry_dia"] == 100]
        df150 = df_valid[df_valid["dry_dia"] == 150]
        df200 = df_valid[df_valid["dry_dia"] == 200]
        df250 = df_valid[df_valid["dry_dia"] == 250]

        time50_i = df50["time"].to_numpy()
        chi50_i = df50["chi"].to_numpy()
        chi50 = np.append(chi50,chi50_i)
        # print("chi50",chi50)
        # print(len(chi50))
        # print("len of chi50",len(chi50))

        chi100_i = df100["chi"].to_numpy()
        chi100 = np.append(chi100,chi100_i)
        # print("chi100",chi100)8210
        # print("len of chi100",len(chi100))
        chi150_i = df150["chi"].to_numpy()
        chi150 = np.append(chi150,chi150_i)

        chi200_i = df200["chi"].to_numpy()
        chi200 = np.append(chi200,chi200_i)

        chi250_i = df250["chi"].to_numpy()
        chi250 = np.append(chi250,chi250_i)

        plt.figure(figsize=(5,4))
        plt.plot(df50.time, chi50, label='Chi 50', marker='o')
        plt.plot(df100.time, chi100, label='Chi 100', marker='o')
        plt.plot(df150.time, chi150, label='Chi 150', marker='o')
        plt.plot(df200.time, chi200, label='Chi 200', marker='o')
        plt.plot(df250.time, chi250, label='Chi 250', marker='o')
        date_str = f"{date:8}"  # Convert to string for formatting
        formatted_date = f"{date_str[0:4]}-{date_str[4:6]}-{date_str[6:8]}"  # YYYY-MM-DD
        plt.title(f'Chi Values Over Time On {formatted_date}')
        plt.xlabel('Time')
        plt.ylabel('Chi Value')
        plt.ylim(0, 1)
        plt.legend()
        plt.grid(True)
        
        plot_path = f'/Users/evenhou/Documents/images/{formatted_date}_chi.png'
        plot_name = f'{formatted_date}_chi.png'
         # Save the plot
        plt.savefig(plot_path)

        # plt.show()
        plt.close()
        #print(plot_name)

## Setup

In [105]:
# Define the folder path
directory = '/Users/evenhou/Downloads/HOU_data'

# Get a list of all files in the folder
files = os.listdir(directory)

# Filter the list to include only files containing "xxxx"
acsm_files = [file for file in files if 'acsm' in file]
acsm_files.sort() # modify the original array
print(len(acsm_files))

htdma_files = [file for file in files if 'htdma' in file]
htdma_files.sort() # modify the original array
print(htdma_files)
print(len(htdma_files))


366
['houaoshtdmaM1.b1.20210913.000950.nc', 'houaoshtdmaM1.b1.20210914.000018.nc', 'houaoshtdmaM1.b1.20210915.000217.nc', 'houaoshtdmaM1.b1.20210916.000418.nc', 'houaoshtdmaM1.b1.20210917.000030.nc', 'houaoshtdmaM1.b1.20210918.000231.nc', 'houaoshtdmaM1.b1.20210919.000430.nc', 'houaoshtdmaM1.b1.20210920.000045.nc', 'houaoshtdmaM1.b1.20210921.000244.nc', 'houaoshtdmaM1.b1.20210922.000445.nc', 'houaoshtdmaM1.b1.20210923.000057.nc', 'houaoshtdmaM1.b1.20210924.000258.nc', 'houaoshtdmaM1.b1.20210925.000459.nc', 'houaoshtdmaM1.b1.20210926.000112.nc', 'houaoshtdmaM1.b1.20210927.000312.nc', 'houaoshtdmaM1.b1.20210928.000511.nc', 'houaoshtdmaM1.b1.20210929.000125.nc', 'houaoshtdmaM1.b1.20210930.000325.nc', 'houaoshtdmaM1.b1.20211001.000525.nc', 'houaoshtdmaM1.b1.20211002.000138.nc', 'houaoshtdmaM1.b1.20211003.000339.nc', 'houaoshtdmaM1.b1.20211004.000538.nc', 'houaoshtdmaM1.b1.20211005.000152.nc', 'houaoshtdmaM1.b1.20211006.000351.nc', 'houaoshtdmaM1.b1.20211007.142057.nc', 'houaoshtdmaM1.b1.20

In [106]:
type(htdma_files)

list

## Test

In [107]:
plot_k_pdf(htdma_files)

/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/1321923889.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_k = pd.concat([df_k, temp_df], ignore_index=True)
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/1321923889.py:66: RuntimeWarning: invalid value encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time, :]/Ntot[i_time]
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/1321923889.py:66: RuntimeWarning: invalid value encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time, :]/Ntot[i_time]
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/1321923889.py:66: RuntimeWarning: invalid value encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time,

In [108]:
plot_acsm(acsm_files)

In [109]:
plot_chi(htdma_files)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/3720744629.py:76: RuntimeWarning: divide by zero encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time, :]/N_tot[i_time]
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/3720744629.py:76: RuntimeWarning: invalid value encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time, :]/N_tot[i_time]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/3720744629.py:131: RuntimeWarning: divide by zero encountered in log
  H_gamma = -p_LH*np.log(p_LH)-p_MH*np.log(p_MH

In [110]:
chi_list = htdma_files
acsm_list = acsm_files
kpdf_list = htdma_files

In [111]:
def save_plot_descriptions(chi_list, acsm_list, kpdf_list, output_filename="latex_code.txt"):
    # Open the output file in write mode
    with open(output_filename, 'w') as f:
        # Loop through chi list
        for i, name in enumerate(chi_list):
            plot_name = plot_chi(chi_list)
            f.write("\includegraphics[scale=0.4]{"+plot_name+"}")

        # Loop through acsm list
        for i, name in enumerate(acsm_list):
            plot_name = plot_acsm(acsm_list)
            f.write("\includegraphics[scale=0.4]{"+plot_name+"}")

        # Loop through kpdf list
        for i, name in enumerate(kpdf_list):
            plot_name = plot_k_pdf(kpdf_list)
            f.write("\includegraphics[scale=0.4]{"+plot_name+"}")

    print(f"Plot descriptions saved to {output_filename}.")


In [112]:
save_plot_descriptions(chi_list, acsm_list, kpdf_list)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/3720744629.py:76: RuntimeWarning: divide by zero encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time, :]/N_tot[i_time]
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/3720744629.py:76: RuntimeWarning: invalid value encountered in divide
  conc_norm[i_time, :] = conc_norm0[i_time, :]/N_tot[i_time]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/n8/bjc7cpvs3hnd3g97l3ky6q4w0000gn/T/ipykernel_29032/3720744629.py:131: RuntimeWarning: divide by zero encountered in log
  H_gamma = -p_LH*np.log(p_LH)-p_MH*np.log(p_MH

KeyboardInterrupt: 